In [138]:
import folium
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
from IPython.display import display
from IPython.display import Image 

route_df = pd.read_csv("54cddc7f1031cfba5dbd4120a081a9fb.csv")
# route_df = pd.read_csv("coyote-chamisoso.csv")

print(route_df.head())
elem = route_df.iloc[0]
startLat = elem['latitude']
startLon = elem['longitude']
numPoints = route_df.shape[0]
print(numPoints)

def find_centroid(route_df):
    centroid_lat = 0
    centroid_lon = 0
    for _, row in route_df.iterrows():
        lat = row['latitude']
        lon = row['longitude']
        centroid_lat += lat
        centroid_lon += lon
    mid = numPoints/2
    centroid_lat = centroid_lat / numPoints
    centroid_lon = centroid_lon / numPoints
    return (centroid_lat, centroid_lon)

centroid = find_centroid(route_df)
print(centroid)
route_map = folium.Map(
    location=centroid,
    zoom_start=14,
    tiles='OpenStreetMap',
    width=1000,
    height=500)

folium.CircleMarker(
    location=[centroid[0], centroid[1]],
    radius=3,
).add_to(route_map)
for _, row in route_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3,
    ).add_to(route_map)
display(route_map)

    latitude   longitude  elevation
0  39.779346 -105.190483    1734.72
1  39.780119 -105.190375    1735.97
2  39.780367 -105.190564    1737.58
3  39.780616 -105.190824    1739.53
4  39.780927 -105.190851    1741.71
332
(39.78054161746988, -105.20376327710837)


In [143]:
from pyproj import Geod
centroid = find_centroid(route_df)
print(centroid)
#OpenTopoMap
route_map = folium.Map(
    location=centroid,
    zoom_start=14,
    tiles='OpenStreetMap')

coordinates = [tuple(x) for x in route_df[['latitude', 'longitude']].to_numpy()]
print(coordinates[0])
print(len(coordinates))
start = coordinates[0]
end = coordinates[-1]
endLon = end[1]
print(f"End lon = {endLon}")
print(f"End lon + 0.5 = {endLon-0.005}")
end = (end[0]-0.0009, (end[1]-0.0009))
start = (start[0], (start[1]+0.0006))

pairs = [(coordinates[idx], coordinates[idx-1]) for idx, val in enumerate(coordinates) if idx != 0]
print(len(pairs))
geodesic = Geod(ellps='WGS84')
rotations = [geodesic.inv(pair[0][1], pair[0][0], pair[1][1], pair[1][0])[0]+90 for pair in pairs]
print(len(rotations))
newRotations = []
newPairs = []
for i in range(0, len(rotations), 15):
    newPairs.append(pairs[i])
    newRotations.append(rotations[i])
print(rotations[:10])

# create the start and end icons
angle = 0
kw1 = {"prefix": "fa", "color": "green", "icon": "play"}
kw2 = {"prefix": "fa", "color": "red", "icon": "stop"}
startIcon = folium.Icon(angle=angle, **kw1)
stopIcon = folium.Icon(angle=angle, **kw2)

# set the start and end markers
folium.Marker(
    location=start,
    color = "green",
    radius=4,
    icon=startIcon
).add_to(route_map)
folium.Marker(
    location=end,
    color = "red",
    radius=4,
    icon=stopIcon
).add_to(route_map)

folium.PolyLine(coordinates, weight=4).add_to(route_map)
kw = {"prefix": "fa", "color": "green", "icon": "play"}
for pair, rot in zip(newPairs, newRotations):
    # icon = folium.Icon(angle=rot, **kw)
    # folium.Marker(location=pair[0], icon=icon, tooltip=str(rot)).add_to(route_map)

    folium.RegularPolygonMarker(location=pair[0], color='black',
        fill=True, fill_color='red', fill_opacity=0.2, icon=icon,
        number_of_sides=2, radius=7, rotation=rot).add_to(route_map)
display(route_map)

(39.78054161746988, -105.20376327710837)
(39.779346, -105.190483)
332
End lon = -105.190483
End lon + 0.5 = -105.195483
331
331
[-83.84723743165722, 239.54416624230623, 231.1434243936154, 266.1678120684429, 224.74449677583263, 217.99147118112643, 226.8210366525685, 265.9469965488088, 202.6896229676263, 94.15869927876872]
